In [18]:
import xml.etree.ElementTree as etree

from itertools import islice

In [ ]:
# http://boscoh.com/programming/reading-xml-serially.html

In [19]:
drugbank_xml = 'resources/drugbank_all_full_database.xml'
drugbank_schema = 'resources/drugbank.xsd'

In [41]:
# first few (two?) elements seems like header or similar
iterable= \
    islice(etree.iterparse(drugbank_xml, 
                           events=('start', 'end', 'start-ns', 'end-ns')), 
           2, 500)

for event, elem in iterable:
    if 'atc' in elem.tag:
        print(event, elem, elem.tag, elem.attrib)

start <Element '{http://www.drugbank.ca}atc-codes' at 0x7f98503bf1d8> {http://www.drugbank.ca}atc-codes {}
start <Element '{http://www.drugbank.ca}atc-code' at 0x7f98503bf228> {http://www.drugbank.ca}atc-code {'code': 'B01AE02'}
end <Element '{http://www.drugbank.ca}atc-code' at 0x7f98503bf228> {http://www.drugbank.ca}atc-code {'code': 'B01AE02'}
end <Element '{http://www.drugbank.ca}atc-codes' at 0x7f98503bf1d8> {http://www.drugbank.ca}atc-codes {}
